In [3]:
import os, os.path as osp
import pandas as pd
from tqdm import tqdm
import datetime
import shutil

scene_name_list = [
    "apple",
    "block",
    "paper-windmill",
    "space-out",
    "spin",
    "teddy",
    "wheel",
]

# root = "../data/iphone_1x_dev/"
root = "../data/iphone_1x_dev/"

# get curent time stamp
backup_root = "../data/iphone_dev_eval_backup"
# name = "iphone_single_ref.yaml"
name="native_iphone.yaml_dep=gt_gt_cam=False"

backup_flag=False


for tto_flag in [True]:
    prefix = "tto_" if tto_flag else ""
    backup_dir = osp.join(
        backup_root, prefix + name + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    )
    os.makedirs(backup_dir, exist_ok=True)

    results = {}
    for scene_name in tqdm(scene_name_list):
        # find the latest log
        log_dirs = os.listdir(osp.join(root, scene_name, "log"))
        log_dirs = [d for d in log_dirs if name in d]

        log_dirs.sort()
        log_dir = log_dirs[-1]
        fn = osp.join(root, scene_name, "log", log_dir, f"{prefix}dycheck_metrics.xlsx")
        assert osp.exists(fn), f"{fn} does not exist"

        # read the first line of xls
        df = pd.read_excel(fn).to_numpy()

        psnr = float(df[0][-3])
        ssim = float(df[0][-2])
        lpips = float(df[0][-1])
        print(scene_name)
        print(fn)
        print(df[0])
        print(psnr, ssim, lpips)
        results[f"{scene_name}-psnr"] = psnr
        results[f"{scene_name}-ssim"] = ssim
        results[f"{scene_name}-lpips"] = lpips
        
        if backup_flag:
            # copy the log_dir into the backup
            shutil.copytree(
                osp.join(root, scene_name, "log", log_dir, f"{prefix}test"),
                osp.join(backup_dir, prefix + scene_name + f"{log_dir}_test"),
            )
            # shutil.copytree(
            #     osp.join(root, scene_name, "log", log_dir, "test_viz"),
            #     osp.join(backup_dir, prefix + scene_name + f"{log_dir}_test_viz"),
            # )
    df = pd.DataFrame([results], index=[0])
    df.to_excel(
        osp.join(backup_dir, f"{prefix}{name}_iphone_collected_masked_metrics.xlsx")
    )

 29%|██▊       | 2/7 [00:00<00:00, 18.07it/s]

apple
../data/iphone_1x_dev/apple/log/native_iphone.yaml_dep=gt_gt_cam=False_20240512_015329/tto_dycheck_metrics.xlsx
['AVE' 0 0 0 13.4395374480943 0.6644279751226418 0.618406714760661]
13.4395374480943 0.6644279751226418 0.618406714760661
block
../data/iphone_1x_dev/block/log/native_iphone.yaml_dep=gt_gt_cam=False_20240512_024610/tto_dycheck_metrics.xlsx
['AVE' 0 0 0 18.28102301121473 0.6756969005807765 0.2278027339305908]
18.28102301121473 0.6756969005807765 0.2278027339305908


AssertionError: ../data/iphone_1x_dev/paper-windmill/log/native_iphone.yaml_dep=gt_gt_cam=False_20240512_041746/tto_dycheck_metrics.xlsx does not exist